In [23]:
import tenseal as ts
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

th.manual_seed(73)

In [24]:
train_X = th.load("data-2/train_X.pt")
train_y = th.load("data-2/train_y.pt")
test_X = th.load("data-2/test_X.pt")
test_y = th.load("data-2/test_y.pt")

In [25]:
# Training dataset
train_dataset = TensorDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size=64)
# Test dataset
test_dataset = TensorDataset(test_X, test_y)
test_loader = DataLoader(test_dataset, batch_size=1)

In [26]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 12)
        
    def forward(self, x):
        out = self.fc1(x)
        out = out * out
        out = self.fc2(out)
        return out

In [27]:
def train(model, device, train_loader, optimizer, criterion, epochs):
    losses = []
    for epoch in range(1, epochs + 1):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        model.eval()
        print('Train Epoch: {:2d}   Avg Loss: {:.6f}'.format(epoch, th.mean(th.tensor(losses))))

    return model

In [28]:
model = Model()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
device = th.device("cuda" if th.cuda.is_available() else "cpu")

device = th.device("cpu")

model = train(model, device, train_loader, optimizer, criterion, 30)

Train Epoch:  1   Avg Loss: 0.681389
Train Epoch:  2   Avg Loss: 0.617955
Train Epoch:  3   Avg Loss: 0.546425
Train Epoch:  4   Avg Loss: 0.497110
Train Epoch:  5   Avg Loss: 0.462913
Train Epoch:  6   Avg Loss: 0.437628
Train Epoch:  7   Avg Loss: 0.417854
Train Epoch:  8   Avg Loss: 0.401683
Train Epoch:  9   Avg Loss: 0.387986
Train Epoch: 10   Avg Loss: 0.376063
Train Epoch: 11   Avg Loss: 0.365469
Train Epoch: 12   Avg Loss: 0.355911
Train Epoch: 13   Avg Loss: 0.347191
Train Epoch: 14   Avg Loss: 0.339169
Train Epoch: 15   Avg Loss: 0.331745
Train Epoch: 16   Avg Loss: 0.324841
Train Epoch: 17   Avg Loss: 0.318395
Train Epoch: 18   Avg Loss: 0.312357
Train Epoch: 19   Avg Loss: 0.306684
Train Epoch: 20   Avg Loss: 0.301341
Train Epoch: 21   Avg Loss: 0.296295
Train Epoch: 22   Avg Loss: 0.291520
Train Epoch: 23   Avg Loss: 0.286990
Train Epoch: 24   Avg Loss: 0.282685
Train Epoch: 25   Avg Loss: 0.278587
Train Epoch: 26   Avg Loss: 0.274677
Train Epoch: 27   Avg Loss: 0.270942
T

In [29]:
def compute_labels(out):
    out = th.sigmoid(out)
    return (out >= 0.5).int()


# compute accuracy using hamming loss
def accuracy(output, target):
    # convert to labels
    out = compute_labels(output)
    # flatten and compute hamming loss
    flat_out = out.flatten()
    flat_target = target.flatten()
    incorrect = th.logical_xor(flat_out, flat_target).sum().item()
    hamming_loss = incorrect / len(flat_out)
    return 1 - hamming_loss


print("Accuracy on test set: {:.2f}".format(accuracy(model(test_X), test_y)))

Accuracy on test set: 0.93


Below is for encrypted model using PyTorch-like model, but which uses TenSEAL operations.

In [36]:
class HEModel:
    def __init__(self, fc1, fc2):
        self.fc1_weight = fc1.weight.t().tolist()
        self.fc1_bias = fc1.bias.tolist()
        self.fc2_weight = fc2.weight.t().tolist()
        self.fc2_bias = fc2.bias.tolist()
        
    def forward(self, encrypted_vec):
        # first fc layer + square activation function
        encrypted_vec = encrypted_vec.mm(self.fc1_weight) + self.fc1_bias
        encrypted_vec *= encrypted_vec
        # second fc layer
        encrypted_vec = encrypted_vec.mm(self.fc2_weight) + self.fc2_bias
        return encrypted_vec
    
    def __call__(self, x):
        return self.forward(x)

In [31]:
# Parameters
bits_scale = 25
coeff_mod_bit_sizes = [30, bits_scale, bits_scale, bits_scale, 30]
polynomial_modulus_degree = 8192

# Create context
context = ts.context(ts.SCHEME_TYPE.CKKS, polynomial_modulus_degree, coeff_mod_bit_sizes=coeff_mod_bit_sizes)
# Set global scale
context.global_scale = 2 ** bits_scale
# Generate galois keys required for matmul in ckks_vector
context.generate_galois_keys()

he_model = HEModel(model.fc1, model.fc2)

Below is Encrypted evaluation of entire dataset.
Steps:
1. encrypt the vector
2. do encrypted evaluation
3. decrypt the result



In [22]:
# how many labels in the encrypted evaluation are the same as in the plain evaluation?
match = 0
he_outs = []
for data, _ in test_loader:
    # remove batch axis, we only need a flat vector
    vec = data.flatten()
    # encryption
    encrypted_vec = ts.ckks_vector(context, vec)
    # encrypted evaluation
    encrypted_out = he_model(encrypted_vec)
    # decryption
    he_out = th.tensor(encrypted_out.decrypt())
    he_outs.append(he_out.tolist())
    out = model(data)
    # how many labels match
    he_labels = compute_labels(he_out)
    plain_labels = compute_labels(out)
    match += (he_labels == plain_labels).sum().item()

KeyboardInterrupt: 

In [ ]:
print("Accuracy on test set (encrypted evaluation): {:.2f}".format(accuracy(th.tensor(he_outs), test_y)))
print("Encrypted evaluation matched {:.1f}% of the labels from the plain evaluation".format(
    match / (12 * len(test_loader)) * 100)
)

For 2 party implementation

-send the encrypted vector for remote evaluation

-then send back encrypted resul for decryption